<a href="https://colab.research.google.com/github/aniketkumar2808/GenAI/blob/main/query_RoutingDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain_community tiktoken langchain_google_genai langchain-cohere langchainhub chromadb langchain langgraph  tavily-python pypdf

In [ ]:
## RAG Implementations
### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata

# Set Gemini API key from secrets
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")
# Set embeddings
embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Docs to index
# Load a research paper and split it into chunks
loader = PyPDFLoader("/content/sample_data/NIPS-2017-attention-is-all-you-need-Paper.pdf")  # Replace with the path to your PDF
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(data)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embd,
)
retriever = vectorstore.as_retriever()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
!pip install -qU mcp-use agno langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.6/870.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 16.7 MB/s eta 0:00:00


In [ ]:
import mcp
from mcp.client.streamable_http import streamablehttp_client
import json
import base64

config = {
  "perplexityApiKey": "aniket"
}
# Encode config in base64
config_b64 = base64.b64encode(json.dumps(config).encode()).decode()
smithery_api_key = "aa86bb28-820c-411c-9ac4-b2ef3bd79af1"

# Create server URL
url = f"https://server.smithery.ai/@arjunkmrm/perplexity-search/mcp?config={config_b64}&api_key={smithery_api_key}"

async def main():
    # Connect to the server using HTTP client
    async with streamablehttp_client(url) as (read_stream, write_stream, _):
        async with mcp.ClientSession(read_stream, write_stream) as session:
            # Initialize the connection
            await session.initialize()
            # List available tools
            tools_result = await session.list_tools()
            print(f"Available tools: {', '.join([t.name for t in tools_result.tools])}")

import sys
if 'ipykernel' in sys.modules:
    import asyncio
    await main()
else:
    import asyncio
    asyncio.run(main())

Available tools: search


In [ ]:
import asyncio
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage
from mcp_use import MCPAgent, MCPClient

# Create MCPClient from configuration dictionary
client = MCPClient.from_dict(config)

# Create LLM
# Initialize Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro",)

# Create agent with the client
SystemMessage(content="""\
                    You are a research assistant powered by Perplexity. Help users find accurate and
                    up-to-date information on any topic they're interested in.

                    - Provide well-structured, comprehensive answers
                    - Cite sources when appropriate
                    - Organize complex information clearly
                    - Be objective and present multiple perspectives when relevant
                    - Use headings and bullet points for better readability
                    - Provide a link to the source from where the information is retreived\
                """)
agent = MCPAgent(llm=llm, client=client, max_steps=30)

# Run the query
# result = await agent.run(
#     "Give me most relevant research news on AI agents in robotics that came in past 3 months.",
# )
# print(f"\nResult: {result}")

In [ ]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from pydantic import BaseModel, Field


# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search", "perplexityMCP"] = Field(
        ...,
        description="Given a user question choose to route it to web search or perplexity MCP or a vectorstore.",
    )


# LLM with function call
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to a vectorstore or web search or Perplexity MCP Server.
The vectorstore contains documents related to Transformer model, attention mechanism,encoders and decoders.
Use the vectorstore for questions on these topics. If the user wants to know about the detailed and contextually relevant
results on any research with citation, then use perplexity MCP. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Who won IPL in 2025?"}
    )
)
print(question_router.invoke({"question": "What is a transformer model?"}))
print(question_router.invoke({"question": "Give me most relevant research news on AI agents in robotics that came in past 3 months."}))

datasource='web_search'
datasource='vectorstore'
datasource='perplexityMCP'
